# Fetch Data

Note book where data is fetched and saved as a .csv to be used for all publication analyses.

Stopped here- do imports, investigate the data and save out. Consider if wanting to use a spoke only, cpoke growth only or probe only df as well.


# 0. Imports


In [1]:
import config as c
import utils as u
import behav_viz.ingest as ingest
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2


[2025-03-23 11:50:23,613][INFO]: Connecting jbreda@bdata00.pni.princeton.edu:3306
[2025-03-23 11:50:24,985][INFO]: Connected jbreda@bdata00.pni.princeton.edu:3306


# 1. Trial DataFrame

Each row of the df frame is a unique trial for a given session, animal. Note that for V2 animals in the growth phase, a trial collapses failed settling in pokes into a summary (e.g. `avg_failed_cpoke_dur`) and represents the final successful trial in a potential bout of pokes. For a fully unraveled DataFrame, see the `Poke DataFrame`


In [ ]:
# load data for a given data range
tdf = ingest.create_trials_df.create_trials_df_from_dj(
    c.ANIMALS, c.DATE_MIN, c.DATE_MAX
)

# clean up dates
tdf = ingest.drop_dates(tdf, c.DATE_DROPS)  # see config.py for rationale

# remove post probe stages
tdf = tdf.query("stage < 11")

# remove columns not relevant to the analysis
tdf = tdf.drop(columns=c.DROP_COLUMNS)

# save data
tdf.to_parquet("trials_df.parquet", index=False)
tdf.head()

animal_id R040
Fetched 75 dates from Sessions table from 2024-07-20 to 2024-10-17 for R040

animal_id R041
Fetched 84 dates from Sessions table from 2024-07-20 to 2024-10-17 for R041

animal_id R042
Fetched 84 dates from Sessions table from 2024-07-20 to 2024-10-17 for R042

animal_id R043
Fetched 85 dates from Sessions table from 2024-07-20 to 2024-10-17 for R043

animal_id R044
Fetched 50 dates from Sessions table from 2024-07-20 to 2024-09-21 for R044

animal_id R045
Fetched 83 dates from Sessions table from 2024-07-20 to 2024-10-17 for R045

animal_id R046
Fetched 84 dates from Sessions table from 2024-07-20 to 2024-10-17 for R046

animal_id R047
Fetched 78 dates from Sessions table from 2024-07-20 to 2024-10-17 for R047

animal_id R048
Fetched 74 dates from Sessions table from 2024-07-27 to 2024-10-17 for R048

animal_id R049
Fetched 69 dates from Sessions table from 2024-07-29 to 2024-10-17 for R049

animal_id R050
Fetched 75 dates from Sessions table from 2024-07-27 to 2024-10-1

,trial,animal_id,date,sess_id,protocol,result,hits,violations,temperror,hit_rate,...,r_water_vol,ab_l_prob,ab_r_prob,ab_beta,ab_tau,exp_tau,exp_min,exp_max,min_time_to_spoke,violation_due_to_spoke
0,1,R040,2024-07-20,963806,FixationGrower,1,1,0,0,0.0,...,60.0,1.0,0.0,0.0,30.0,0.5,0.1,1.2,7.6683,NaN
1,2,R040,2024-07-20,963806,FixationGrower,1,1,0,0,1.0,...,60.0,1.0,0.0,0.0,30.0,0.5,0.1,1.2,5.1108,NaN
2,3,R040,2024-07-20,963806,FixationGrower,1,1,0,0,1.0,...,60.0,1.0,0.0,0.0,30.0,0.5,0.1,1.2,3.1533,NaN
3,4,R040,2024-07-20,963806,FixationGrower,6,<NA>,<NA>,<NA>,1.0,...,60.0,1.0,0.0,0.0,30.0,0.5,0.1,1.2,17.7304,NaN
4,5,R040,2024-07-20,963806,FixationGrower,6,<NA>,<NA>,<NA>,1.0,...,60.0,1.0,0.0,0.0,30.0,0.5,0.1,1.2,12.0591,NaN


In [5]:
t = u.load_trials_df()

Loading trials data from directory: /Users/jessbreda/Desktop/github/jbreda_animal_training/notebooks/fixation_publication


In [17]:
t.columns

Index(['trial', 'animal_id', 'date', 'sess_id', 'protocol', 'result', 'hits',
       'violations', 'temperror', 'hit_rate', 'error_rate', 'violation_rate',
       'temp_error_rate', 'n_lpokes', 'n_cpokes', 'n_rpokes', 'trial_dur',
       'trial_start', 'first_spoke', 'first_lpoke', 'first_rpoke',
       'n_settling_ins', 'avg_settling_in', 'cpoke_dur', 'stage', 'stage_name',
       'curriculum', 'fix_experiment', 'sides', 'SMA_set',
       'auto_spoke_to_cpoke', 'settling_in_dur', 'legal_cbreak_dur',
       'delay_dur', 'pre_go_dur', 'inter_trial_dur', 'fixation_dur',
       'initial_fixation_dur', 'target_fixation_dur', 'fixation_adjustment',
       'has_reached_target_fixation', 'settling_in_determines_fixation',
       'delay_growth', 'delay_warm_up_on', 'settling_in_growth',
       'settling_in_warm_up_on', 'warm_up_imp', 'go_type', 'go_dur',
       'give_type_set', 'give_type_imp', 'give_frac', 'timeouts',
       'water_delivered', 'give_water_not_drunk', 'crash_hist',
       'was

# 2. Days DataFrame

Each row of the df frame is data from a unique day for a given animal and focus on measures like mass, rig used, total water drunk etc.


In [37]:
ddf = ingest.create_days_df.create_days_df_from_dj(
    animal_ids=c.ANIMALS,
    date_min=c.DATE_MIN,
    date_max=c.DATE_MAX,
    verbose=True,
)

ddf = ingest.drop_dates(ddf, c.DATE_DROPS)

Fetched data from SessionAggDate table from 2024-07-20 to 2024-10-17  
Today is not being queried or data already exists in the database.
No new data from today, 2025-03-03 to add to SessionAggDate.

1365 daily summaries fetched for animals: 
['R040', 'R041', 'R042', 'R043', 'R044', 'R045', 'R046', 'R047', 'R048', 'R049', 'R050', 'R051', 'R052', 'R053', 'R054', 'R055', 'R056', 'R057']
between 2024-07-20 and 2024-10-17
dropping R042 2024-07-29 length: 1365
length: 1364
dropping R043 2024-07-26 length: 1364
length: 1363
dropping R043 2024-07-27 length: 1363
length: 1362
dropping R043 2024-07-28 length: 1362
length: 1361
dropping R043 2024-07-29 length: 1361
length: 1360
dropping R046 2024-08-12 length: 1360
length: 1359
dropping R052 2024-08-14 length: 1359
length: 1358
dropping R051 2024-08-20 length: 1358
length: 1357
dropping R051 2024-08-21 length: 1357
length: 1356
dropping R051 2024-08-22 length: 1356
length: 1355


In [49]:
# Add in stage and fix_experiment information from the trial df & remove post-prone trials (which get by left joining to tdf)

tdf_agg = (
    tdf.groupby(["date", "animal_id"])
    .agg({"stage": "first", "fix_experiment": "first"})
    .reset_index()
)
tdf_agg["date"] = pd.to_datetime(tdf_agg["date"])

ddf_merged = pd.merge(tdf_agg, ddf, on=["date", "animal_id"], how="left")

# save data
ddf_merged.to_parquet("days_df.parquet", index=False)
ddf_merged.head()

,date,animal_id,stage,fix_experiment,session_agg_date_id,num_sessions,n_done_trials,rigid,starttime,endtime,...,num_water,rig_volume,num_rigwater,train_dur_hrs,trial_rate,starttime_hrs,endtime_hrs,side_bias,volume_target,water_diff
0,2024-07-20,R040,1.0,V1,3447340,1,18,Rig01,0 days 15:44:27,0 days 16:01:21,...,1,0.744,1,0.281667,63.91,15.740833,16.022500,-0.99,98.6,-56.528
1,2024-07-20,R041,1.0,V2,3447341,1,147,Rig04,0 days 13:49:28,0 days 16:01:23,...,1,2.670,1,2.198611,66.86,13.824444,16.023056,-0.97,96.2,-93.530
2,2024-07-20,R042,1.0,V1,3447342,1,163,Rig05,0 days 13:49:29,0 days 16:01:18,...,1,8.400,1,2.196944,74.19,13.824722,16.021667,-0.99,105.8,-97.400
3,2024-07-20,R043,1.0,V2,3447343,1,148,Rig06,0 days 13:49:28,0 days 16:01:20,...,1,2.874,1,2.197778,67.34,13.824444,16.022222,-0.89,97.0,-94.126
4,2024-07-20,R044,1.0,V1,3447344,1,144,Rig09,0 days 13:49:33,0 days 16:01:21,...,1,1.896,1,2.196667,65.55,13.825833,16.022500,-0.96,80.0,-78.104


## 3. Poke DataFrame


In [2]:
import glob
import pandas as pd

# Define the folder path
fpath = "/Volumes/brody/jbreda/FixationGrower_cpoke_table/"

# Use glob to find all CSV files in the folder
csv_files = glob.glob(fpath + "*.csv")

# Read each CSV file into a DataFrame and store them in a list
df_list = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames into a single DataFrame
pdf = pd.concat(df_list, ignore_index=True)

# Display the first few rows of the combined DataFrame
pdf.head()


,sessid,animal_id,trial,cpoke_dur,cpoke_iti,post_settling_violation,settling_violation,was_rewarded,fixation_dur,date,stage
0,964851,R043,1,0.3827,NaN,0,1,0,0.401,30-Jul-2024 14:47:25,5.0
1,964851,R043,1,0.1890,0.5006,0,1,0,0.401,30-Jul-2024 14:47:25,5.0
2,964851,R043,1,0.3638,0.8218,0,1,0,0.401,30-Jul-2024 14:47:25,5.0
3,964851,R043,1,0.3416,0.2565,0,1,0,0.401,30-Jul-2024 14:47:25,5.0
4,964851,R043,1,0.0580,0.2194,0,1,0,0.401,30-Jul-2024 14:47:25,5.0


In [3]:
def assign_fix_experiment(animal_id):
    if animal_id in c.V1_ANIMALS:
        return "V1"
    elif animal_id in c.V2_ANIMALS:
        return "V2"
    return None


# Apply mapping & make additional columns
pdf["fix_experiment"] = pdf["animal_id"].map(assign_fix_experiment)
pdf["relative_cpoke_dur"] = pdf["cpoke_dur"] - pdf["fixation_dur"]
pdf["was_valid"] = (pdf["post_settling_violation"] == 0) & (
    pdf["settling_violation"] == 0
)

# clean up funky things from the imperfect matlab scrape
pdf.loc[pdf["cpoke_iti"] < 0, "cpoke_iti"] = pd.NA
pdf.loc[pdf["cpoke_iti"] > 7200, "cpoke_iti"] = pd.NA


Now, we need to do some additional cleaning, specifically for the iti variable. We will create a new column called "cpoke_tit_timeout_adj" which is the trial initiation time that has the violation penalty removed.


In [4]:
TIMEOUT_PENALTY_DUR = 2

# growing stages- only need to remove penalty time for V2 animals 
pdf_growing_stages = pdf.query("stage in @c.GROWING_STAGES").copy()
pdf_growing_stages['cpoke_tit_timeout_adj'] = np.where(
    pdf_growing_stages['fix_experiment'] == 'V2',
    pdf_growing_stages['cpoke_iti'],
    np.where(
        pdf_growing_stages['post_settling_violation'],
        pdf_growing_stages['cpoke_iti'] - TIMEOUT_PENALTY_DUR,
        pdf_growing_stages['cpoke_iti']
    )
)

POST_GROWTH_STAGES = [8,9,10]
pdf_post_growth = pdf.query("stage in @POST_GROWTH_STAGES and settling_violation==False").copy()
pdf_post_growth['cpoke_tit_timeout_adj'] = np.where(
        pdf_post_growth['post_settling_violation'],
        pdf_post_growth['cpoke_iti'] - TIMEOUT_PENALTY_DUR,
        pdf_post_growth['cpoke_iti']
)

pdf_cleaned = pd.concat([pdf_growing_stages, pdf_post_growth], ignore_index=True)


In [5]:
pdf_cleaned = pdf_cleaned.query("cpoke_tit_timeout_adj >0") # deal with occasional bpod timing issues

# Prev History
pdf_cleaned = pdf_cleaned.sort_values(by=["animal_id", "date", "trial"])
pdf_cleaned["prev_was_valid"] = (
    pdf_cleaned
    .groupby(["animal_id", "date"])["was_valid"]
    .shift(1)
)

In [6]:

# save data
pdf_cleaned.to_parquet("poke_df.parquet", index=False)